In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import numpy as np
import pickle
import os, cv2
import pandas as pd

Loaded the model fully trained on 14000 training images

In [0]:
model = load_model("/content/drive/My Drive/model114kbatch.h5")

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 480, 640, 24)      1824      
_________________________________________________________________
batch_normalization_1 (Batch (None, 480, 640, 24)      96        
_________________________________________________________________
activation_1 (Activation)    (None, 480, 640, 24)      0         
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 240, 320, 24)      0         
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 240, 320, 36)      21636     
_________________________________________________________________
batch_normalization_2 (Batch (None, 240, 320, 36)      144       
_________________________________________________________________
activation_2 (Activation)    (None, 240, 320, 36)      0         
__________

In [32]:
print(len(os.listdir("/content/drive/My Drive/test_images")))

12815


We convert each image into an ndarray and append these arrays to the list X_data which will be used for getting predictions.
The size of the list containing all 12815 images would be around 10Gb which cannot be processed into our RAM that's why we separated the testing images into batches of 2000 images like we did with training images and got the predictions.

In [ ]:
X = []

for i in range(1,2001):
    myFile = "/content/drive/My Drive/test_images/" + str(i) + ".png"
    if i%100==0:
        print(myFile)
    image = cv2.imread(myFile)
    X.append (image)

Saved this list into the drive in case we need this list in future.

In [0]:
with open("/content/drive/My Drive/2ktest_batch.txt", "wb") as fp:
    pickle.dump(X, fp)

In [ ]:
X = np.array(X)

In [ ]:
X = X.astype(np.float32)
X.dtype

In [ ]:
mean2 = X[:,:,:,2].mean() 
std2=X[:,:,:,2].std()
mean1 = X[:,:,:,1].mean() 
std1=X[:,:,:,1].std()
mean = X[:,:,:,0].mean() 
std=X[:,:,:,0].std()
X[:,:,:,0] = (X[:,:,:,0] - mean)/std
X[:,:,:,1] = (X[:,:,:,1] - mean1)/std1
X[:,:,:,2] = (X[:,:,:,2] - mean2)/std2

Got the prediction for the first batch of test images

In [ ]:
out=model.predict(X)
out

Saved this prediction array into the drive

In [0]:
with open("/content/drive/My Drive/2ktestpred_batch.txt", "wb") as fp:
    pickle.dump(out, fp)

In [0]:
del model #clear the RAM
del X

All previous steps are repeated until we get predictions for bounding boxes of all 12815 test images
Finally we load the predictions of all the images and stack them together into a single array

In [0]:
with open("/content/drive/My Drive/2ktestpred_batch.txt", "rb") as fp:   # Unpickling
    pred1 = pickle.load(fp)

In [0]:
with open("/content/drive/My Drive/4ktestpred_batch.txt", "rb") as fp:   # Unpickling
    pred2 = pickle.load(fp)

In [0]:
with open("/content/drive/My Drive/6ktestpred_batch.txt", "rb") as fp:   # Unpickling
    pred3 = pickle.load(fp)

In [0]:
with open("/content/drive/My Drive/8ktestpred_batch.txt", "rb") as fp:   # Unpickling
    pred4 = pickle.load(fp)

In [0]:
with open("/content/drive/My Drive/10ktestpred_batch.txt", "rb") as fp:   # Unpickling
    pred5 = pickle.load(fp)

In [0]:
with open("/content/drive/My Drive/12ktestpred_batch.txt", "rb") as fp:   # Unpickling
    pred6 = pickle.load(fp)

In [0]:
with open("/content/drive/My Drive/lasttestpred_batch.txt", "rb") as fp:   # Unpickling
    pred7 = pickle.load(fp)

In [0]:
boxprediction = np.concatenate((pred1,pred2,pred3,pred4,pred5,pred6,pred7) , axis = 0)

In [96]:
boxprediction.shape , boxprediction.dtype

((12815, 4), dtype('float32'))

After getting all the predictions we make the csv file

In [95]:
test = pd.read_csv('/content/drive/My Drive/test (2).csv')
test.head(5)

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,NaN,NaN,NaN,NaN
1,1473231475010DeeplearnS11276.png,NaN,NaN,NaN,NaN
2,JPEG_20161205_135307_1000155917326.png,NaN,NaN,NaN,NaN
3,JPEG_20160711_123440_1000518778437.png,NaN,NaN,NaN,NaN
4,JPEG_20160803_115329_100034020722.png,NaN,NaN,NaN,NaN


In [97]:
boxprediction = boxprediction.astype(np.int64)
boxprediction


array([[214, 435,  92, 389],
       [ 88, 554, 114, 364],
       [117, 510,  46, 431],
       ...,
       [ 87, 508,  52, 423],
       [191, 498, 115, 367],
       [ 39, 597,  78, 396]])

In [98]:
boxprediction.dtype

dtype('int64')

In [0]:
df2 = test.copy()

In [0]:
df2['x1'] = boxprediction[:,0]

In [0]:
df2['x2'] = boxprediction[:,1]

In [0]:
df2['y1'] = boxprediction[:,2]
df2['y2'] = boxprediction[:,3]

In [112]:
df2.head(10)

,image_name,x1,x2,y1,y2
0,1474723840903DSC08089.png,214,435,92,389
1,1473231475010DeeplearnS11276.png,88,554,114,364
2,JPEG_20161205_135307_1000155917326.png,117,510,46,431
3,JPEG_20160711_123440_1000518778437.png,194,461,80,400
4,JPEG_20160803_115329_100034020722.png,62,571,43,431
5,147444974116511473239803010-Mast--Harbour-Men-...,149,481,64,413
6,JPEG_20160622_110649_1000527459853.png,146,507,102,378
7,JPEG_20160823_120737_1000784898268.png,102,490,69,406
8,1480965956645IMG_3094.png,168,481,61,418
9,14732348976417a565e40d545452688130062b267a2d2.png,40,598,23,450


In [0]:
df2.to_csv('/content/drive/My Drive/testpred3.csv', index=False)